In [3]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import numpy as np
import pandas as pd

Parte 1: Carga de datos.
Acá se cargarán los datos de matches.csv .


In [4]:
df = pd.read_csv("matches.csv")

C:\Users\paulm\AppData\Local\Temp\ipykernel_18068\2538391591.py:1: DtypeWarning: Columns (136,141,143,145,146,148,149,151,152,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("matches.csv")


Parte 2: Preprocesamiento de datos.

In [5]:
df.head()

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_bench_10,home_bench_10_minute,home_bench_11_num,home_bench_11,home_bench_11_minute,away_bench_10_num,away_bench_10,away_bench_10_minute,away_starting_14_num,away_starting_14
0,18123,Charlton Athletic,Everton,"Saturday, August 18",2001,14:00,"20,451","The Valley, London, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18124,Derby County,Blackburn Rovers,"Saturday, August 18",2001,14:00,"28,236","Pride Park Stadium, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18125,Leeds United,Southampton,"Saturday, August 18",2001,14:00,"39,715","Elland Road, Leeds, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18126,Leicester City,Bolton Wanderers,"Saturday, August 18",2001,14:00,"19,987","Filbert Street, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18127,Liverpool,West Ham United,"Saturday, August 18",2001,14:00,"43,935","Anfield, Liverpool, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Especificar las columnas deseadas
columnas_deseadas = ['home_wonCorners', 'away_wonCorners', 'home_redCards', 'away_redCards',
                     'home_yellowCards', 'away_yellowCards', 'home_shotsSummary', 'away_shotsSummary',
                     'home_possessionPct', 'away_possessionPct', 'home_score', 'away_score',
                     'home', 'away']

# Filtrar el dataframe manteniendo solo las columnas deseadas
df = df.filter(items=columnas_deseadas)

df = df.dropna()


In [7]:
df.head()

,home_wonCorners,away_wonCorners,home_redCards,away_redCards,home_yellowCards,away_yellowCards,home_shotsSummary,away_shotsSummary,home_possessionPct,away_possessionPct,home_score,away_score,home,away
0,0.0,0.0,0.0,0.0,0.0,3.0,0 (0),0 (0),0%,0%,1,2,Charlton Athletic,Everton
1,0.0,0.0,0.0,0.0,1.0,0.0,0 (0),0 (0),0%,0%,2,1,Derby County,Blackburn Rovers
2,0.0,0.0,0.0,1.0,1.0,2.0,0 (0),0 (0),0%,0%,2,0,Leeds United,Southampton
3,0.0,0.0,0.0,0.0,4.0,3.0,0 (0),0 (0),0%,0%,0,5,Leicester City,Bolton Wanderers
4,0.0,0.0,0.0,0.0,1.0,5.0,0 (0),0 (0),0%,0%,2,1,Liverpool,West Ham United


In [8]:
df['result'] = np.where(df['home_score'] > df['away_score'], 'Home Win', np.where(df['home_score'] < df['away_score'], 'Away Win', 'Draw'))

In [9]:
df.head()

,home_wonCorners,away_wonCorners,home_redCards,away_redCards,home_yellowCards,away_yellowCards,home_shotsSummary,away_shotsSummary,home_possessionPct,away_possessionPct,home_score,away_score,home,away,result
0,0.0,0.0,0.0,0.0,0.0,3.0,0 (0),0 (0),0%,0%,1,2,Charlton Athletic,Everton,Away Win
1,0.0,0.0,0.0,0.0,1.0,0.0,0 (0),0 (0),0%,0%,2,1,Derby County,Blackburn Rovers,Home Win
2,0.0,0.0,0.0,1.0,1.0,2.0,0 (0),0 (0),0%,0%,2,0,Leeds United,Southampton,Home Win
3,0.0,0.0,0.0,0.0,4.0,3.0,0 (0),0 (0),0%,0%,0,5,Leicester City,Bolton Wanderers,Away Win
4,0.0,0.0,0.0,0.0,1.0,5.0,0 (0),0 (0),0%,0%,2,1,Liverpool,West Ham United,Home Win


In [10]:
# Extraer los tiros al arco del equipo local
df['home_shotsSummary'] = df['home_shotsSummary'].str.extract(r'\((\d+)\)').astype(int)

# Extraer los tiros al arco del equipo visitante
df['away_shotsSummary'] = df['away_shotsSummary'].str.extract(r'\((\d+)\)').astype(int)

# Extraer el valor de la posesión del equipo local
df['home_possessionPct'] = df['home_possessionPct'].str.extract(r'(\d+)').astype(int)

# Extraer el valor de la posesión del equipo visitante
df['away_possessionPct'] = df['away_possessionPct'].str.extract(r'(\d+)').astype(int)


# Obtener el tipo de datos de la columna 'column_name'
tipo_dato = df['home_possessionPct'].dtype

# Imprimir el tipo de datos
print(tipo_dato)


int32


In [11]:
# Crear un diccionario de mapeo de equipos a ID
equipo_id_map = {}

# Asignar un ID único a cada equipo
for equipo in df['home'].unique():
    equipo_id_map[equipo] = len(equipo_id_map) + 1

# Agregar una nueva columna 'home_id' con el ID del equipo local
df['home_id'] = df['home'].map(equipo_id_map)

# Agregar una nueva columna 'away_id' con el ID del equipo visitante
df['away_id'] = df['away'].map(equipo_id_map)


In [12]:

df.drop('home', axis=1, inplace=True)
df.drop('away', axis=1, inplace=True)

df.head()

,home_wonCorners,away_wonCorners,home_redCards,away_redCards,home_yellowCards,away_yellowCards,home_shotsSummary,away_shotsSummary,home_possessionPct,away_possessionPct,home_score,away_score,result,home_id,away_id
0,0.0,0.0,0.0,0.0,0.0,3.0,0,0,0,0,1,2,Away Win,1,11
1,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,2,1,Home Win,2,16
2,0.0,0.0,0.0,1.0,1.0,2.0,0,0,0,0,2,0,Home Win,3,17
3,0.0,0.0,0.0,0.0,4.0,3.0,0,0,0,0,0,5,Away Win,4,12
4,0.0,0.0,0.0,0.0,1.0,5.0,0,0,0,0,2,1,Home Win,5,18


In [19]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


# Dividir los datos en conjunto de entrenamiento y prueba
X = df.drop('result', axis=1)
y = df['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros que deseas probar en Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9, 11],
    'criterion': ['gini', 'entropy']
}

# Crear el modelo de Random Forest
rf_model = RandomForestClassifier()

# Realizar la búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(rf_model, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus resultados
best_rf_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Mostrar los resultados del mejor modelo
print("Mejor modelo de Random Forest:")
print(f"Best parameters: {best_params}")
print(f"Best score: {best_score:.4f}")
print("\n")

# Obtener los resultados de todos los modelos evaluados en GridSearchCV
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Model with parameters: {params}")
    print(f"Mean test score: {mean_score:.4f}")
    print("\n")


Mejor modelo de Random Forest:
Best parameters: {'criterion': 'gini', 'max_depth': 11, 'n_estimators': 200}
Best score: 0.9947


Model with parameters: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 100}
Mean test score: 0.7529


Model with parameters: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 200}
Mean test score: 0.7606


Model with parameters: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 300}
Mean test score: 0.7600


Model with parameters: {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 100}
Mean test score: 0.9389


Model with parameters: {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 200}
Mean test score: 0.9440


Model with parameters: {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 300}
Mean test score: 0.9521


Model with parameters: {'criterion': 'gini', 'max_depth': 7, 'n_estimators': 100}
Mean test score: 0.9865


Model with parameters: {'criterion': 'gini', 'max_depth': 7, 'n_estimators': 200}
Mean test score: 0.9900


Model w

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Entrenar el modelo con los mejores parámetros
best_rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Realizar predicciones en el conjunto de entrenamiento
y_train_pred = best_rf_model.predict(X_train)

# Calcular métricas en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

# Mostrar las métricas en el conjunto de entrenamiento
print("Métricas del mejor modelo de Random Forest en el conjunto de entrenamiento:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-score: {train_f1:.4f}")

# Calcular métricas en el conjunto de prueba (ya calculadas anteriormente)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred, average='weighted')
test_recall = recall_score(y_test, y_pred, average='weighted')
test_f1 = f1_score(y_test, y_pred, average='weighted')

# Mostrar las métricas en el conjunto de prueba
print("Métricas del mejor modelo de Random Forest en el conjunto de prueba:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-score: {test_f1:.4f}")



Métricas del mejor modelo de Random Forest en el conjunto de entrenamiento:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000
Métricas del mejor modelo de Random Forest en el conjunto de prueba:
Accuracy: 0.9918
Precision: 0.9919
Recall: 0.9918
F1-score: 0.9918
